# Day 2: Recap of Python fundamentals (continued)

In today's tutorial, we'll cover:
- File handling;
- Recursion;
- Memoisation; and
- Object-oriented programming.

A set of exercises that will allow you to test your learning of this tutorial will also be made available.  

## Using files to store data

So far, we've either hard-coded data into our programs, or we've allowed the user to enter it using their keyboard (with `input()`). Of course, if we want to work with larger data sets, we'll need a better way of getting data into and out of our programs. Python allows us to read and write to _files_ stored in the computer's _filesystem_.

Storing data as files has a number of benefits. Files are persistent, so the data they contain remains even when your computer is switched off. They're also easy to share with others, allowing data to be uploaded for use by others. Finally, the computer's filesystem provides a built-in way of organising files into folders.

If you've worked with files in other programming languages, you'll know that it can sometimes be tricky. We first need to open the file, by giving a _path_ to its location. We can then read and write to the file. And finally, we need to make sure that we _close_ the file once we're finished, so that all of the changes we make are saved properly.

Python, however, makes dealing with files much more straightforward:

In [2]:
with open("data/phonebook.txt") as phonebookFile:
    print(phonebookFile.readline())

Alice,123456789



In the first line of this example, we've told Python to open the file called `phonebook.txt`, using the `open` function. The `open` function returns an object that represents the file, and `as phonebookFile` assigns this to the variable called `phonebookFile`. We haven't covered objects in this track, but for now all we need to know is that we can perform operations (like reading and writing) on the file via the object assigned to `phonebookFile`. Finally, the `with` statement opens a block where it is valid to access the file via `phonebookFile`. The file will be closed once the `with` block has finished. That means that the above example safely opens the file, reads a line from it, and closes it.

As we can see from the above example, we can read the file line-by-line using the `readline` method. However, while this can sometimes be useful, we often want to read everything in the file. We can do this all at once:

In [3]:
with open("data/phonebook.txt") as phonebookFile:
    print(phonebookFile.read())

Alice,123456789
Bob,987654321
Maude,987462513



The `read` method gives us all of the data in the file at once. It is much more common that we'd want to go through the file line-by-line, and do something with each entry:

In [3]:
with open("data/phnebook.txt") as phonebookFile:
    for entry in phonebookFile:
        print(entry)

FileNotFoundError: [Errno 2] No such file or directory: 'data/phnebook.txt'

In this example, we've used the `for` loop to go through the entire file. The loop variable `entry` will take the value of each line of the file in turn. As we can see, the data isn't quite formatted in the way that we'd like. There are a couple of issues: there's a new-line character at the end of each line, causing the spacing that we see above. In addition, each entry is run together - we might want to make the name and phone number look neater.

To solve the spacing issue, we can remove the newline character:

In [7]:
fruits = ["apples","oranges","bananas","melons"]
prices = [20,10,5,15]
quantities = [5,7,3,4]

for fruit, price, quantity in zip(fruits,prices,quantities):
     print(f"You bought {quantity} {fruit} for ${price*quantity}")

You bought 5 apples for $100
You bought 7 oranges for $70
You bought 3 bananas for $15
You bought 4 melons for $60


In [4]:
with open("data/phonebook.txt") as phonebookFile:
    for entry in phonebookFile:
        entry = entry.strip()
        print(entry)

Alice,123456789
Bob,987654321
Maude,987462513


The `strip` method removes any whitespace from the start and end of the string, including regular spaces, tabs, and newline characters.

Next, we can separate the name from the phone number:

In [5]:
with open("data/phonebook.txt") as phonebookFile:
    for entry in phonebookFile:
        name, phone_number = entry.strip().split(',')
        print(f"{name} can be reached at {phone_number}.")

Alice can be reached at 123456789.
Bob can be reached at 987654321.
Maude can be reached at 987462513.


Here, we've used `strip` to remove the newline character as before. Next, though, we've used the `split` method. The `split` method will divide the string into a tuple, where each entry was separated by the character that we pass to the method. So, in this example, each line contained a name followed by a comma followed by a phone number. `split` will produce a tuple that has the name as the first element, and the phone number as the second element. We then assign these values to the `name` and `phone_number` variables as shown.

Notice how we can chain different method calls together: we read these left-to-right. So, we first apply the `strip` method, and then apply the `split` method to the result.

It is common that we will use files to store data, and then read the data from the file into a data structure that we can use throughout our program. Thinking back to yesterday's dictionary example, we can build the phonebook data structure from the data stored in the file:

In [6]:
#instantiate an empty dictionary
phonebook = {}

with open("data/phonebook.txt") as phonebookFile:
    for entry in phonebookFile:
        name, phone_number = entry.strip().split(',')
        #Add entry into the dictionary with name as the key
        phonebook[name] = phone_number

for name, phone_number in phonebook.items():
    print(f"{name} can be reached at {phone_number}.")
print(phonebook)

Alice can be reached at 123456789.
Bob can be reached at 987654321.
Maude can be reached at 987462513.
{'Alice': '123456789', 'Bob': '987654321', 'Maude': '987462513'}


As shown, we first instantiate an empty dictionary, and assign this to the variable `phonebook`. We can then build up the dictionary, adding an entry to the phonebook for each line in the file. Once we've read in all the data, we can use the dictionary as usual.

Of course, we might want to make changes to our data structure, and have these changes saved back to the file:

In [12]:
phonebook = {}

# open the phonebook data file, and read the entries into `phonebook`
with open("data/phonebook.txt") as phonebookFile:
    for entry in phonebookFile:
        name, phone_number = entry.strip().split(',')
        phonebook[name] = phone_number

# add a new entry
phonebook["Claude"] = "1029384756"

# save the phonebook back to the file
with open("data/phonebook.txt", "w") as phonebookFile:
    for name, phone_number in phonebook.items():
        #this line only saves the new input to the file
        print(f"{name},{phone_number}", file=phonebookFile)

OSError: [Errno 30] Read-only file system: 'data/phonebook.txt'

In this example, we construct the phonebook from the data stored in the file, as in the previous example. Next, however, we add a new entry to the phonebook for `Claude`. Once we've made that change to the phonebook, we then want to save the new phonebook to the data file. To do that, we use the same `with` block, but this time, we pass an additional parameter to the `open` function. The `w` indicates that we want to open the file for *w*riting, which we need to do if we want to change the file. Next, inside the `with` block, we iterate through the phonebook, and print each entry in turn. Notice that we specify a `file` when we call the `print` function: this means that, instead of printing to the screen, the output will be added to the file we've specified.

To check that we've been successful, we can re-run one of the earlier examples, printing out the phonebook from the data file:

In [13]:
with open("data/phonebook.txt") as phonebookFile:
    for entry in phonebookFile:
        name, phone_number = entry.strip().split(',')
        print(f"{name} can be reached at {phone_number}.")

Alice can be reached at 123456789.
Bob can be reached at 987654321.
Maude can be reached at 987462513.


## Recursion

Yesterday, we went over how we can use functions to break down a problem, and solve it using small blocks of code. However, sometimes we encounter problems that have solutions that are defined in terms of themselves. For example, we might want to compute the _factorial_ of a number. The factorial of a number, _n_, is defined as the number itself, multiplied by each of the natural numbers less than it. That means that the factorial of _n_ is _n_ multiplied by the factorial of _n - 1_.

So, we could define a function to calculate the factorial in Python:

In [14]:
def factorial(n):
    return n * factorial(n-1)

factorial(3)

RecursionError: maximum recursion depth exceeded

Now, this example code follows our definition of factorial: `factorial(n)` returns `n * factorial(n-1)`. However, it doesn't work: when we try to run the code above (with the example `factorial(3)`, we get an exception. That's because our function will try to run forever: `factorial(3) = 3 * factorial(2) = 3 * 2 * factorial(1) = 3 * 2 * 1 * factorial(0) = 3 * 2 * 1 * 0 * factorial(-1)`, and so on. There's nothing in our code that stops it from continuing to go down through all of the negative numbers - right down to infinity. Python stops us from running our code after a certain point, if it detects that we've written something that might go on forever. That limit is what gives us the error message above.

To stop this from happening, we need to explicitly define the _base case_. In recursive definitions, the base case a value for which the function is not defined in terms of itself, and where it can return a known value. For our `factorial` function, this is when `n` is 1: `factorial(1)` is 1:

In [15]:
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

factorial(3)

6

Now when we run this code, we get the expected answer: 6. If we hand-execute the code, we can see that we arrive at the base case, and that we won't run indefinitely.

So, whenever we encounter a problem that we think we can solve recursively, we should always think about the base cases.

Let's look at another example. In the 13th century, an Italian mathematician, Fibonacci, attempted to model the growth of a rabbit population. His model resulted in the _Fibonacci sequence_ which defines the _n_th number as the sum of the previous two numbers in the sequence: so, `F(n) = F(n-2) + F(n-1)`. Of course, we need a base case: in fact, given that the sequence is defined in terms of two values of itself, we need two base cases. These are the 0th number in the sequence, which equals 0, and the 1st number, which equals 1.

Now, we can write our Python code to solve this problem:

In [16]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-2) + fibonacci(n-1)

fibonacci(10)

55

If we hand-executed this code, we'd find that it gives us the right answer: 55. 

In summary, when we can define the solution to a problem in terms of itself, then we can use recursive programming to solve the problem. However, we need to be careful that we have one or more base cases, to ensure that the program will terminate.

## Memoisation

In some programs, we need to repeatedly use the result of some computation. If those computations are expensive (i.e., they take a long time), then we would want to avoid repeating the computation. 

Let's think about the calculation of the Fibonacci sequence that we described above. Because the nth number in the sequence is defined in terms of the previous two numbers in the sequence, we end up recomputing the early values in the sequence. To illustrate this, let's add a `print` statement to our code above:

In [17]:
def fibonacci(n):
    print(f"Working out the {n}th Fibonacci number..")
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-2) + fibonacci(n-1)

fibonacci(4)

Working out the 4th Fibonacci number..
Working out the 2th Fibonacci number..
Working out the 0th Fibonacci number..
Working out the 1th Fibonacci number..
Working out the 3th Fibonacci number..
Working out the 1th Fibonacci number..
Working out the 2th Fibonacci number..
Working out the 0th Fibonacci number..
Working out the 1th Fibonacci number..


3

Here, we print a statement every time we call the `fibonacci` function. We've only calculated the first four numbers of the Fibonacci sequence for readability, but even with this small number, we can see that we calculate values repeatedly. Imagine what this might look like for really large numbers - we end up recomputing the previous numbers in the sequence lots of times.

So, instead of calculating the number, we could _remember_ the result of the computation, and fetch the stored result as needed. This is called _memoisation_. We can do this quite easily, using the data structures we are already familiar with:

In [18]:
fibonacci_numbers = {}

def memoised_fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        if n not in fibonacci_numbers:
            fibonacci_numbers[n] = memoised_fibonacci(n-2) + memoised_fibonacci(n-1)
        return fibonacci_numbers[n]

memoised_fibonacci(20)

6765

In this version, we've added a dictionary, `fibonacci_numbers`, where we'll store the Fibonacci sequence as we create it. Inside our `memoised_fibonacci` function, we calculate a Fibonacci number only when we haven't done so before. In this way, we avoid the repeated computations.

The savings of using memoisation can sometimes seem quite abstract. To try and illustrate them, we're going to time how long it takes to calculate a reasonably big Fibonacci number. We'll use Python's `time` module to do this, first timing the unmemoised version (without the `print` statement, for fairness):

In [19]:
import time

def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-2) + fibonacci(n-1)

start = time.time()
print(fibonacci(35))
end = time.time()
print(f"fibonacci took {end-start} seconds!")

9227465
fibonacci took 2.745462417602539 seconds!


.. and then the memoised version (with an empty dictionary, again, for fairness):

In [20]:
fibonacci_numbers = {}

def memoised_fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        if n not in fibonacci_numbers:
            fibonacci_numbers[n] = memoised_fibonacci(n-2) + memoised_fibonacci(n-1)
        return fibonacci_numbers[n]

start = time.time()
print(memoised_fibonacci(35))
end = time.time()
print(f"memoised fibonacci took {end-start:.10f} seconds!")

9227465
memoised fibonacci took 0.0001237392 seconds!


We can see that the memoised version of our function was much quicker than the non-memoised version. Indeed, this can have very significant consequences on the runtime of our program. We can use the memoised version of our function to calculate really large Fibonacci numbers: try the snippet above with the number 5000, for example. But for our non-memoised version, the time taken to calculate each number in the sequence increases exponentially: you'll be there for a long time if you use it to calculate the 5000th Fibonacci number.

While `time` is useful to show what's going on in a general sense, there are a lot of factors that can affect the results we get, including the hardware of the computer we're using, and the other software running on the computer at the same time. In track 2 next week, we'll see a better, more scientific approach to reasoning about time complexity.

Memoisation essentially swaps time complexity for space complexity: storing all of those Fibonacci numbers for use later on takes up memory in our computer. We need to think about whether or not this is a worthwhile trade-off, depending on the problems we want to solve.

## Object-oriented programming

Python is a _multi-paradigm_ language: it lets us write programs that mix-and-match different programming styles, like procedural, with others like object-oriented and functional programming. Today, we'll recap object-oriented programming, and tomorrow, we'll look at Python's functional programming support.

Object-oriented programming lets us structure our program around _objects_: these bundle data and the computations that we can perform on that data together. Objects represents things, and behaviours of those things. For example, we might have a car object, that stores data (like the model or registration number), and has operations, like `drive`, that let us do things with those objects.

Objects are instantiations of _classes_. We can define a class in Python like this:

In [21]:
class Car:
    def __init__(self, model, registration_number):
        self.model = model
        self.registration_number = registration_number

Before we explain what this code does, let's first instantiate a car object, of the `Car` class:

In [22]:
my_car = Car("Ferarri", "R3A11Y F4ST")

Now, when we defined the `Car` class, we defined a function within it: the `__init__` function. Functions that are defined inside classes are called _methods_. The `__init__` method is always called when we instantiate an object: so, when we defined `my_car`, and instantiated a `Car` object, we called its `__init__` method.

If we look at the definition of the `__init__` method, we can see that it takes 3 arguments: `self`, `model`, and `registration_number`. However, when we instantiated the class, we only gave it a model and registration number: we're missing an argument! That's because as soon as instantiate the class, we create an instance, and that instance is passed to the `__init__` method as the `self` argument.

Inside the `__init__` method, we set two _attributes_: `model` and `registration_number`. The attributes that we set here are specific to the instance of the `Car` class, and we set them from the values we pass in to the `__init__` method. We can inspect the values of an object's attributes:

In [23]:
print(my_car.model)

Ferarri


We can also change the value of these attributes:

In [24]:
my_car.model = "Toyota"
print(my_car.model)

Toyota


Now, of course, we might not want to allow arbitrary changes to an object's attributes. We do this with two main changes to our code:

In [30]:
class Car:
    def __init__(self, model, registration_number):
        self.model = model
        self._registration_number = registration_number
    
    def change_registration_number(self, new_registration_number):
        if new_registration_number.startswith("R"):
            #raise Exception(f"Sorry - you can't change the registration number to {new_registration_number}")
            print(f"Sorry - you can't change the registration number to {new_registration_number}")
        else:
            self._registration_number = new_registration_number

my_car2 = Car("Toyota", "N0T R3A11Y F4ST")

In this version of our `Car` class, we've made two changes. First, when we create the `registration_number` attribute, we've put an underscore in front of it (making it `_registration_number`). This is a convention that signals that we don't want code to directly access the attribute. Next, we've added a `change_registration_number` method. That means that we expect any changes to `_registration_number` to go through that method. This is useful for performing checks on updates. In this example, we make sure that we can't update the registration number to anything beginning with R:

In [31]:
my_car2.change_registration_number("H3LL0")

.. will work as expected, but:

In [32]:
my_car2.change_registration_number("R3A11Y F4ST")

Sorry - you can't change the registration number to R3A11Y F4ST


will throw an exception because the new value begins with R.

In this tutorial, we've recapped the very basics of object-oriented programming. Next week, in track 2, we'll revisit object-oriented programming in much more depth.

## Summary

In this tutorial, we've recapped:
- File handling;
- Recursion;
- Memoisation; and
- Object-oriented programming.